In [ ]:
pip install pandas openpyxl

In [ ]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.withdraw()

import os
import openpyxl
import pandas as pd
import time

# Input

In [ ]:

filepath_topics = filedialog.askopenfilename(title="Select file listing Topics (projects)")
print(filepath_topics)

sheet_name = 'Topics'

try:
    df_topics = pd.read_excel(filepath_topics, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")
    

In [ ]:

filepath_students = filedialog.askopenfilename(title="Select file listing Students and their preferences")
print(filepath_students)

# Read 'Students' sheet
sheet_name = 'Students'

try:
    df_students = pd.read_excel(filepath_students, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")


'''
# Read 'Students preferences' sheet
sheet_name = 'Student_preferences'

try:
    df_students_preferences = pd.read_excel(filepath_students, sheet_name=sheet_name)
    #df_preferences = pd.read_excel(filepath_student_preferences, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")

df_students_preferences.rename(columns={'Your name (Lastname, First name)' : 'TypedName',
                        'Have you submitted a Thesis Self-Proposal?': 'Self-proposal',
                        'Do you wish to work in a pair': 'Pair',
                        'Please enter the name of your partner': 'Partner',
                        'Please select the project code of your 1st preferred research Topic': 'Topic1',
                        'Please select the project code of your 2nd preferred research Topic': 'Topic2',
                        'Please select the project code of your 3rd preferred research Topic': 'Topic3',
                        'Have you had a meaningful discussion with the supervisors of those first three research Topics you have selected (by email, Teams or in-person)?': 'Discussion',
                        'Please select the project code of your 4th preferred research Topic': 'Topic4', 
                        'Please select the project code of your 5th preferred research Topic': 'Topic5',
                        'Please, select your 1st preferred Subject Area that you would be happy to work in (see Thesis Handbook for details).' : 'SubjectArea1',
                        'Please, select your 2nd preferred Subject Area that you would be happy to work in.' : 'SubjectArea2',
                        'Please, select your 3rd preferred Subject Area that you would be happy to work in.' : 'SubjectArea3',
                        'Please, select your 1st preferred Research Type that you would be happy to work on.' : 'ProjectType1',
                        'Please, select your 2nd preferred Research Type that you would be happy to work on.' : 'ProjectType2'},
                         inplace = True)

print(df_students_preferences.columns) 
'''

# Read 'Students self_proposals' sheet
sheet_name = 'Student_selfproposals'

try:
    data_selfproposals = pd.read_excel(filepath_students, sheet_name=sheet_name)
    #data_selfproposals = pd.read_excel(filepath_selfproposals, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")


In [ ]:

filepath_allocations = filedialog.askopenfilename(title="Select file listing the Allocation optimisation results")
print(filepath_allocations)

# Read the 'Student_allocations' sheet
sheet_name = 'Student_allocations'

try:
    df_student_allocations = pd.read_excel(filepath_allocations, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")

    
# Read the 'Supervisor_allocations' sheet
sheet_name = 'Supervisor_allocations'

try:
    df_supervisor_allocations = pd.read_excel(filepath_allocations, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")
    

# Create Spreadsheet for Students and Supervisors

In [ ]:
# ---------------------------------------------
# Create filepath for the results 
# --------------------------------------------- 
folder_results = os.path.dirname(os.path.abspath(filepath_allocations))

timestr = time.strftime("%Y%m%d-%H%M%S")
file_results = "03_Output_" + timestr + ".xlsx"

filepath_results = os.path.join(folder_results,file_results)
#print (filepath_results)

wb = openpyxl.Workbook()
wb.save(filepath_results)


# ---------------------------------------------
# Create new DataFrame to save preferences as table 
# --------------------------------------------- 

# Define the column names
results_columns = ['Student', 'Email', 'Code', 'Supervisor', 'Title', 'Subject Area', 'Type', 'Source']

print(results_columns)

# Create an empty DataFrame with specified columns
results_df = pd.DataFrame(columns=results_columns)


# Add data to results DataFrame 
# -----------------------------
for index, row in df_students.iterrows():

    student_name = row['Full Name']
    student_email = row['Email Address']
    print(f"Populating result for {student_name} with email {student_email}") 
    
    project_code = "" 
    project_supervisor = ""
    project_title = ""
    project_area = ""
    project_type = ""
    project_source = ""
    
    student_allocation_rows = df_student_allocations.loc[df_student_allocations['Student'] == student_name]
    student_selfproposal_rows = data_selfproposals.loc[data_selfproposals['Student'] == student_name]
    
    if student_allocation_rows.empty == False:
        
        #print("student_allocation_rows: ", student_allocation_rows)
        
        for index2, row2 in student_allocation_rows.iterrows():
            
            print("student allocation row: ", row2)
            
            project_code = row2['Project']
            
            project_rows = df_topics.loc[df_topics['Code'] == project_code]
            
            if project_rows.empty:
                continue

            #print("project_rows: ", project_rows)
    
            for index3, row3 in project_rows.iterrows():
        
                project_supervisor = row3['Supervisor']
                project_title = row3['Title']
                project_area = row3['Subject Area']
                project_type = row3['Research Type']
                project_source = "Directory"

    elif student_selfproposal_rows.empty == False:
        
        #print("student_selfproposal_rows: ", student_selfproposal_rows)
        
        for index2, row2 in student_selfproposal_rows.iterrows():
            
            print("student selfproposal row: ", row2)
            
            project_code = row2['Code']   
            project_supervisor = row2['Supervisor']
            project_title = row2['Title']
            project_area = row2['Subject Area']
            project_type = row2['Type']
            project_source = "Self-proposed"            
    
    
    student_result = pd.DataFrame({'Student' : [student_name],
                                   'Email' : [student_email],
                                   'Code' : [project_code], 
                                   'Supervisor' : [project_supervisor], 
                                   'Title' : [project_title],
                                   'Subject Area' : [project_area],
                                   'Type' : [project_type],
                                   'Source' : [project_source]})
                
    results_df = pd.concat([results_df, student_result], ignore_index=True)

    
if 'NONE' in results_df.columns:
    results_df.drop(columns=['NONE'], inplace=True)

print(results_df)



# Use ExcelWriter to write to the specified filename
with pd.ExcelWriter(filepath_results, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    results_df.to_excel(writer, sheet_name='Final Allocations', index=False)

print(f"Sheet 'Final Allocations' has been successfully written to {filepath_results}")

wb = openpyxl.load_workbook(filepath_results)
print(wb.sheetnames)
defaultSheet = wb['Sheet']
wb.remove(defaultSheet)
wb.save(filepath_results)


